# CNN

In [18]:
# Overall Model architecture

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 300, 16)           2048      
                                                                 
 conv1d_4 (Conv1D)           (None, 296, 128)          10368     
                                                                 
 global_max_pooling1d_4 (Gl  (None, 128)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_8 (Dense)             (None, 320)               41280     
                                                                 
 dense_9 (Dense)             (None, 5)                 1605      
                                                                 
Total params: 55301 (216.02 KB)
Trainable params: 55301 (216.02 KB)
Non-trainable params: 0 (0.00 Byte)
________________

## Imports

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
from google.colab import output#oven timer
import os
from sklearn.metrics import f1_score, accuracy_score

# Pre Proccess

##Load Data

In [2]:
print(os.getcwd())
#Path of the data folder
#data_path = "C:/Users/Admin/OneDrive - McGill University/COMP551550git/NLP-Authorship-Latin-Eng\Data"
data_path = ""
os.path.exists(data_path)

/content


False

In [3]:

# Load your CSV dataset
training_data = pd.read_csv((data_path + "translated_data_train_16k_format_fix.csv"))
validation_data = pd.read_csv((data_path + "validation_data_filtered.csv"))
test_data = pd.read_csv((data_path + "test_data_filtered.csv"))

train_en_x = training_data['en'].values
train_la_x = training_data['la'].values
valid_en_x = validation_data['en'].values
valid_la_x = validation_data['la'].values
test_en_x  = test_data['en'].values
test_la_x  = test_data['la'].values

train_y = training_data['file'].values
valid_y = validation_data['file'].values
test_y = test_data['file'].values

# Encode labels
label_encoder = OneHotEncoder()
label_encoder.fit(train_y.reshape(-1,1))

train_y=label_encoder.transform(train_y.reshape(-1,1)).toarray()
valid_y =label_encoder.transform(valid_y.reshape(-1,1)).toarray()
test_y= label_encoder.transform(test_y.reshape(-1,1)).toarray()


## Tokenize

In [4]:
def tokenize(text_nump, tokenizer, maxlen):
  seq = tokenizer.texts_to_sequences(text_nump)
  return tf.keras.preprocessing.sequence.pad_sequences(seq, maxlen = maxlen)

In [5]:
# Define parameters
max_len = 300  # maximum sequence length
vocab_size = 128  # ASCII characters

# Tokenize and pad the text sequences
tokenizer_la = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters=None, lower=False)
tokenizer_la.fit_on_texts(train_la_x)

tokenizer_en = tf.keras.preprocessing.text.Tokenizer(char_level=True, filters=None, lower=False)
tokenizer_en.fit_on_texts(train_en_x)

train_seq_la = tokenize(train_la_x, tokenizer_la, max_len)
train_seq_en = tokenize(train_en_x, tokenizer_en, max_len)
valid_seq_la = tokenize(valid_la_x, tokenizer_la, max_len)
valid_seq_en = tokenize(valid_en_x, tokenizer_en, max_len)
test_seq_la = tokenize(test_la_x, tokenizer_la, max_len)
test_seq_en= tokenize(test_en_x, tokenizer_en, max_len)


# Model

In [16]:
def make_model(): #match model architecture
  # Build the model
  model = models.Sequential()
  model.add(layers.Embedding(input_dim=vocab_size, output_dim=16, input_length=300))
  model.add(layers.Conv1D(128, 5, activation='relu'))
  model.add(layers.GlobalMaxPooling1D())
  model.add(layers.Dense(320, activation='relu'))
  model.add(layers.Dense(len(label_encoder.categories_[0]), activation='softmax'))

  # Compile the model
  model.compile(optimizer='adam',
                loss='categorical_crossentropy',
                metrics=['accuracy'])
  return model
def decode_predictions(predictions):
    return predictions.argmax(axis=1)

## Latin Model

In [8]:

# Build the model
model = make_model()
# Train the model
model.fit(train_seq_la, train_y, epochs=10, batch_size=64,
          validation_data = (valid_seq_la, valid_y), validation_batch_size = 64
          )

# Evaluate the model on the test set
test_predictions_la = model.predict(test_seq_la)



Epoch 1/10
273/273 [==============================] - 5s 14ms/step - loss: 1.1298 - accuracy: 0.5168 - val_loss: 0.1540 - val_accuracy: 0.9669
Epoch 2/10
273/273 [==============================] - 1s 5ms/step - loss: 0.6785 - accuracy: 0.7252 - val_loss: 0.1194 - val_accuracy: 0.9725
Epoch 3/10
273/273 [==============================] - 1s 5ms/step - loss: 0.5400 - accuracy: 0.7904 - val_loss: 0.1026 - val_accuracy: 0.9697
Epoch 4/10
273/273 [==============================] - 1s 5ms/step - loss: 0.4606 - accuracy: 0.8233 - val_loss: 0.1137 - val_accuracy: 0.9725
Epoch 5/10
273/273 [==============================] - 2s 6ms/step - loss: 0.3891 - accuracy: 0.8533 - val_loss: 0.1952 - val_accuracy: 0.9532
Epoch 6/10
273/273 [==============================] - 1s 5ms/step - loss: 0.3411 - accuracy: 0.8745 - val_loss: 0.1500 - val_accuracy: 0.9642
Epoch 7/10
273/273 [==============================] - 2s 6ms/step - loss: 0.2955 - accuracy: 0.8909 - val_loss: 0.1572 - val_accuracy: 0.9642
Epoch

[3.1572774e-03 8.0439878e-07 6.2861704e-06 6.3621775e-07 9.9683505e-01]


array([0., 0., 0., 0., 1.])

In [17]:
# Convert predictions and true labels back to original form
decoded_test_predictions_la = decode_predictions(test_predictions_la)
decoded_true_labels = decode_predictions(test_y)

test_accuracy = accuracy_score(decoded_true_labels, decoded_test_predictions_la)
print(f"Test Accuracy: {test_accuracy}")

# Calculate F1 Score
test_f1_score = f1_score(decoded_true_labels, decoded_test_predictions_la, average='weighted')
print(f"Test F1 Score: {test_f1_score}")

Test Accuracy: 0.9805555555555555
Test F1 Score: 0.9814575598709848


## English Model

In [18]:
# Build the model
model = make_model()
# Train the model
model.fit(train_seq_en, train_y, epochs=10, batch_size=64,
          validation_data = (valid_seq_en, valid_y), validation_batch_size = 64
          )

test_predictions_en = model.predict(test_seq_en)


Epoch 1/10
273/273 [==============================] - 5s 13ms/step - loss: 1.2033 - accuracy: 0.4952 - val_loss: 0.5914 - val_accuracy: 0.7796
Epoch 2/10
273/273 [==============================] - 1s 5ms/step - loss: 0.6310 - accuracy: 0.7629 - val_loss: 0.2942 - val_accuracy: 0.8843
Epoch 3/10
273/273 [==============================] - 1s 5ms/step - loss: 0.4389 - accuracy: 0.8423 - val_loss: 0.6309 - val_accuracy: 0.7741
Epoch 4/10
273/273 [==============================] - 1s 5ms/step - loss: 0.3255 - accuracy: 0.8863 - val_loss: 0.3894 - val_accuracy: 0.8402
Epoch 5/10
273/273 [==============================] - 2s 9ms/step - loss: 0.2553 - accuracy: 0.9120 - val_loss: 0.3340 - val_accuracy: 0.8898
Epoch 6/10
273/273 [==============================] - 2s 7ms/step - loss: 0.2004 - accuracy: 0.9320 - val_loss: 0.6418 - val_accuracy: 0.8044
Epoch 7/10
273/273 [==============================] - 2s 6ms/step - loss: 0.1570 - accuracy: 0.9487 - val_loss: 0.5192 - val_accuracy: 0.8292
Epoch

In [19]:
# Convert predictions and true labels back to original form
decoded_test_predictions_en = decode_predictions(test_predictions_en)
decoded_true_labels = decode_predictions(test_y)

test_accuracy = accuracy_score(decoded_true_labels, decoded_test_predictions_en)
print(f"Test Accuracy: {test_accuracy}")

# Calculate F1 Score
test_f1_score = f1_score(decoded_true_labels, decoded_test_predictions_en, average='weighted')
print(f"Test F1 Score: {test_f1_score}")

Test Accuracy: 0.7916666666666666
Test F1 Score: 0.8503940993715884
